In [2]:
using DrWatson
@quickactivate "MEngProject"
using MEngProject

┌ Info: Precompiling MEngProject [d0493a11-efc0-4c7c-9b66-d0bd5a04cc55]
└ @ Base loading.jl:1260
ERROR: LoadError: LoadError: syntax: extra token ")" after end of expression
Stacktrace:
 [1] top-level scope at /mnt/storage_1/users/cullinanen/.julia/dev/MEngProject.jl/src/Parameters.jl:64
 [2] include(::Module, ::String) at ./Base.jl:377
 [3] include(::String) at /mnt/storage_1/users/cullinanen/.julia/dev/MEngProject.jl/src/MEngProject.jl:1
 [4] top-level scope at /mnt/storage_1/users/cullinanen/.julia/dev/MEngProject.jl/src/MEngProject.jl:7
 [5] include(::Module, ::String) at ./Base.jl:377
 [6] top-level scope at none:2
 [7] eval at ./boot.jl:331 [inlined]
 [8] eval(::Expr) at ./client.jl:449
 [9] top-level scope at ./none:3
in expression starting at /mnt/storage_1/users/cullinanen/.julia/dev/MEngProject.jl/src/Parameters.jl:64
in expression starting at /mnt/storage_1/users/cullinanen/.julia/dev/MEngProject.jl/src/MEngProject.jl:7


ErrorException: Failed to precompile MEngProject [d0493a11-efc0-4c7c-9b66-d0bd5a04cc55] to /mnt/storage_1/users/cullinanen/.julia/compiled/v1.4/MEngProject/ixx4J_19dC0.ji.

In [3]:
using CUDA, PyPlot, NNlib,  ImageFiltering, Images, MEngProject, MEngProject.LamKernels, MEngProject.Laminart, MEngProject.Utils, BenchmarkTools, Test

┌ Info: Precompiling MEngProject [d0493a11-efc0-4c7c-9b66-d0bd5a04cc55]
└ @ Base loading.jl:1260
ERROR: LoadError: LoadError: syntax: extra token ")" after end of expression
Stacktrace:
 [1] top-level scope at /mnt/storage_1/users/cullinanen/.julia/dev/MEngProject.jl/src/Parameters.jl:64
 [2] include(::Module, ::String) at ./Base.jl:377
 [3] include(::String) at /mnt/storage_1/users/cullinanen/.julia/dev/MEngProject.jl/src/MEngProject.jl:1
 [4] top-level scope at /mnt/storage_1/users/cullinanen/.julia/dev/MEngProject.jl/src/MEngProject.jl:7
 [5] include(::Module, ::String) at ./Base.jl:377
 [6] top-level scope at none:2
 [7] eval at ./boot.jl:331 [inlined]
 [8] eval(::Expr) at ./client.jl:449
 [9] top-level scope at ./none:3
in expression starting at /mnt/storage_1/users/cullinanen/.julia/dev/MEngProject.jl/src/Parameters.jl:64
in expression starting at /mnt/storage_1/users/cullinanen/.julia/dev/MEngProject.jl/src/MEngProject.jl:7


ErrorException: Failed to precompile MEngProject [d0493a11-efc0-4c7c-9b66-d0bd5a04cc55] to /mnt/storage_1/users/cullinanen/.julia/compiled/v1.4/MEngProject/ixx4J_19dC0.ji.

In [4]:
img = convert(CuArray{Float32,2}, load(datadir("Iine_100_100_gs.png")));
p = LaminartGPU.kernels(img, Parameters.parameters);
p = LaminartGPU.add_I_u_p(img, p);

UndefVarError: UndefVarError: LaminartGPU not defined

In [ ]:
tspan = (0.0f0, 100.0f0)
u0 = reshape(zeros(Float32, p.dim_i, p.dim_j*(5*p.K+2)), p.dim_i, p.dim_j, 5*p.K+2);